In [ ]:
import numpy as np
import elements
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Set Earth As Central Body
# Length Resolution of 100 km
cb = elements.central_body()
unit_length = 1000e3 # To course for good results, but quick!
geo_orbit = 42000e3
phys_prop = elements.link_physical_properties(unit_length=unit_length, diamter=20, density=7900)
cable_mass_geo = geo_orbit * phys_prop.cross_sectional_area * phys_prop.density

# Variation on Cable Length

In [ ]:
plt.figure(figsize=(9,9))
plt.grid()
anchor_mass = 0
for cable_length in np.arange(0.2,4,0.5)*geo_orbit:
    ev = elements.space_evelator(cable_length=cable_length, phys_prop=phys_prop, anchor_mass=anchor_mass)
    plt.plot(ev.radii[:,0], ev.tensions[:,0], '-', label=f'{cable_length/geo_orbit:1.1f} ')
plt.legend(title='$\\frac{cable_length}{Geo}$', ncol=2)

# Variation on Anchor Mass

In [ ]:
plt.figure(figsize=(9,9))
plt.grid()
cable_length = 60000e3
for anchor_mass in np.arange(0,1.6,0.1) * cable_mass_geo:
    ev = elements.space_evelator(cable_length=cable_length, phys_prop=phys_prop, anchor_mass=anchor_mass)
    plt.plot(ev.radii[:,0], ev.tensions[:,0], '-', label=f'{anchor_mass/ev.cable_mass:1.1f}')
plt.legend(title='$\\frac{AnchorMass}{CableMass}$', ncol=2)


In [ ]:
cable_lengths =  np.arange(1.0, 2.5, 0.05) * geo_orbit
aprox_anchor_masses = cable_lengths * phys_prop.cross_sectional_area * phys_prop.density
anchor_masses = np.arange(cable_mass_geo * 0,  cable_mass_geo* 2.1, 0.05*cable_mass_geo)

max_tensile_stress = np.zeros((len(cable_lengths),len(anchor_masses)))
max_compressive_stress = np.zeros((len(cable_lengths),len(anchor_masses)))

for i, cable_length in enumerate(cable_lengths):
    for ii, anchor_mass in enumerate(anchor_masses):
        ev = elements.space_evelator(cable_length=cable_length, phys_prop=phys_prop, anchor_mass=anchor_mass)
        max_tensile_stress[i,ii] = ev.tensions.max() / phys_prop.cross_sectional_area
        max_compressive_stress[i,ii] = ev.tensions.min() / phys_prop.cross_sectional_area

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,9), sharex=True, sharey=True)

massmass, lenlen = np.meshgrid(anchor_masses, cable_lengths)
a = ax[0].pcolormesh(massmass / cable_mass_geo, lenlen / geo_orbit, max_tensile_stress / 1e9, shading='nearest')
ax[0].set_title('Max Tension GPa')
ax[0].set_ylabel('Cable Length / Geo Orbit')
ax[0].set_xlabel('Mass / Geo Cable Mass')
b = ax[1].pcolormesh(massmass / cable_mass_geo, lenlen / geo_orbit, abs(max_compressive_stress / 1e9), shading='nearest')
ax[1].set_title('Max Compression GPa')
fig.colorbar(a, ax=ax[0], label='Tension Gpa')
fig.colorbar(b, ax=ax[1], label='Compression Gpa')